In [74]:
import datetime
import os
from datetime import timedelta
from os.path import abspath
import pandas as pd
from pandas import ExcelWriter
from openpyxl import Workbook

from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

warehouse_location = abspath('spark-warehouse')
os.environ["PYSPARK_SUBMIT_ARGS"] = '--jars /data/jupyter/kudu-spark2_2.11-1.8.0.jar pyspark-shell'

spark = SparkSession.builder \
    .appName("Data completeness check") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.driver.memory", '8g') \
    .config("spark.executor.memory", '8g') \
    .config("spark.num.executors", '4') \
    .config("hive.exec.compress.output", 'false') \
    .enableHiveSupport() \
    .getOrCreate()

sc = spark.sparkContext

sqlc = SQLContext(sc)

In [56]:
pred_insert_date = '20190830'

OOS_CHECK_DATE = '20190911'

In [57]:
kudu_tables = [
    'lfms.daily_dctrxn', 'lfms.daily_dcstock', 'lfms.ord', 'lfms.daily_shipment', 'lddb.calendar'
]

for tbl in kudu_tables:
    spark.read.format('org.apache.kudu.spark.kudu') \
    .option('kudu.master', "dtla1apps11:7051,dtla1apps12:7051,dtla1apps13:7051") \
    .option('kudu.table', f'impala::{tbl}') \
    .load() \
    .registerTempTable('{}'.format(tbl.replace('.', '_')))

In [67]:
def read_query_and_fetch(sql_path, create_table=False, get_query=False, kudu_replace=None, **query_params):
    with open(sql_path, 'r') as f:
        query = f.read()
    if kudu_replace is not None:
        for k, v in kudu_replace.items():
            query = query.replace(k, v)
    if not create_table:
        ## remove lines with `table`
        q0 = query
        query = '\n'.join([line for line in q0.split('\n')
                           if ('drop table' not in line.lower())
                           and ('create table' not in line.lower())])
    query = query.format(**query_params)
    if get_query:
        return query
    return sqlc.sql(query)

In [91]:
oos_item_list_dc = read_query_and_fetch(
    'kpi_oos_item_list_dc.sql',
    database_name='vartefact', oos_check_date=OOS_CHECK_DATE,
    kudu_replace={'lfms.daily_dcstock': 'lfms_daily_dcstock'}
)

In [92]:
print(oos_item_list_dc.count(), "out of stock items in DC")

23 out of stock items in DC


In [93]:
oos_item_list_dc.createOrReplaceTempView("oos_item_list_dc")

In [94]:
order_check_sql = \
    """
with weekly_predict as (
    select
        cal.week_key, pred.item_id, pred.sub_id,
        sum(daily_sales_pred_original) as weekly_pred
    from
        vartefact.forecast_regular_results_week_to_day_original_pred_all pred
        join lddb_calendar cal on pred.date_key = cal.date_key
        join oos_item_list_dc oild
        on oild.item_id = pred.item_id
        and oild.sub_id = pred.sub_id
    where
         pred.insert_date_key = {0}
    group by
        cal.week_key, pred.item_id, pred.sub_id
),
weekly_order as (
    select
        cal.week_key, ord.item_id, ord.sub_id,
        sum(ord.order_qty) as weekly_order
    from
        vartefact.forecast_onstock_orders ord
        join lddb_calendar cal on ord.order_day = cal.date_key
        join oos_item_list_dc oild
        on oild.item_id = ord.item_id
        and oild.sub_id = ord.sub_id
    group by
        cal.week_key, ord.item_id, ord.sub_id
)
select
    prd.item_id,
    prd.sub_id,
    prd.week_key,
    cal2.date_key as week_start_day,
    prd.weekly_pred,
    ord.weekly_order
from
    weekly_predict prd
    join weekly_order ord on prd.week_key = ord.week_key
        and prd.item_id = ord.item_id
        and prd.sub_id = ord.sub_id
    join lddb_calendar cal2 on cal2.week_key = ord.week_key
    and cal2.dow = 2
order by
    prd.sub_id,
    prd.week_key
    """.replace("\n", " ").format(pred_insert_date)

In [95]:
df = sqlc.sql(order_check_sql).toPandas()

In [96]:
df.to_csv(f'{OOS_CHECK_DATE}_output.csv')